# Quantum Mechanics: 1D Infinite Square Well (Particle in a Box)

## Learning Objectives

By the end of this notebook, you should be able to:

- Write down the time-independent Schrödinger equation for a 1D infinite square well.
- State the boundary conditions and analytic eigenvalues/eigenfunctions.
- Construct a finite-difference Hamiltonian matrix for the well.
- Solve the discrete eigenvalue problem numerically and compare with analytic results.
- Visualize the first few eigenstates and their probability densities.
- Explore convergence with respect to grid resolution.

In [1]:
# Cell 1: Imports and global settings

import numpy as np
import matplotlib.pyplot as plt

# For nicer plots (optional)
plt.rcParams["figure.figsize"] = (6, 4)
plt.rcParams["axes.grid"] = True

## Physical Model

We consider a particle of mass $m$ confined in a 1D infinite square well of width $L$:

$$
V(x) = \begin{cases}
0, & 0 < x < L \\
\infty, & \text{otherwise}.
\end{cases}
$$

The time-independent Schrödinger equation  is
$$
\hat{H} = -\frac{\hbar^2}{2m} \frac{\mathrm{d}^2}{\mathrm{d}x^2} + V(x)
$$
The time-independent Schrödinger equation on $(0, L)$ is
$$
\hat{H} \psi(x) = E \psi(x), \qquad
\hat{H} = -\frac{\hbar^2}{2m} \frac{\mathrm{d}^2}{\mathrm{d}x^2},
$$

with boundary conditions

$$
\psi(0) = 0, \quad \psi(L) = 0.
$$

The exact eigenpairs are

$$
E_n = \frac{\hbar^2 \pi^2 n^2}{2mL^2}, \quad
\psi_n(x) = \sqrt{\frac{2}{L}} \sin\!\left(\frac{n\pi x}{L}\right), \quad n=1,2,\dots
$$

We will now reproduce these results numerically.


In [2]:
# Cell 2: Physical constants and problem parameters

hbar = 1.0   # choose convenient units
m    = 1.0
L    = 1.0

print(f"hbar = {hbar}, m = {m}, L = {L}")


hbar = 1.0, m = 1.0, L = 1.0


## Spatial Discretization and Hamiltonian Matrix

We introduce a uniform grid on $[0, L]$:

- Total points: $N$
- Grid points: $x_i = i\,\Delta x$, $i = 0, \dots, N-1$
- Spacing: $\Delta x = \dfrac{L}{N-1}$

The wavefunction is represented as a vector of values $\psi_i \approx \psi(x_i)$.

The infinite potential walls enforce Dirichlet boundary conditions:

$$
\psi_0 = 0, \quad \psi_{N-1} = 0.
$$

We solve only for **internal** points $i = 1, \dots, N-2$.

Second derivative (central difference):

$$
\frac{\mathrm{d}^2 \psi}
     {\mathrm{d}x^2}\Big|_{x_i}
\approx 
\frac{\psi_{i+1} 
- 2 \psi_i + \psi_{i-1}}{\Delta x^2}.
$$

On the internal grid, the second derivative becomes a tridiagonal matrix
$D^{(2)}$. The Hamiltonian operator is

$$
H = -\frac{\hbar^2}{2m} D^{(2)}.
$$

In [3]:
# Build the 1D infinite well Hamiltonian (finite difference)

def make_infinite_well_1d(n_points=200, L=1.0, m=1.0, hbar=1.0):
    """
    Construct the 1D infinite square well Hamiltonian using
    a finite-difference Laplacian with Dirichlet boundary conditions.

    Parameters
    ----------
    n_points : int
        Total grid points including boundaries at x=0 and x=L.
    L : float
        Well width.
    m : float
        Particle mass.
    hbar : float
        Reduced Planck constant.

    Returns
    -------
    x : (n_points,) ndarray
        Full spatial grid including boundaries.
    x_internal : (n_points-2,) ndarray
        Internal spatial grid where the Hamiltonian acts.
    H : (n_internal, n_internal) ndarray
        Hamiltonian matrix on internal points.
    """
    x = np.linspace(0.0, L, n_points)
    dx = x[1] - x[0]

    # Internal points (exclude boundaries)
    x_internal = x[1:-1]
    n_internal = x_internal.size

    # Second derivative matrix with Dirichlet BCs
    diag = np.full(n_internal, -2.0)
    off  = np.ones(n_internal - 1)
    D2 = (np.diag(diag) +
          np.diag(off, k=1) +
          np.diag(off, k=-1)) / dx**2

    # Hamiltonian H = -(ħ^2 / 2m) D2
    H = -(hbar**2 / (2.0 * m)) * D2
    return x, x_internal, H

# Quick sanity check on shape
x_test, x_int_test, H_test = make_infinite_well_1d(n_points=10, L=L, m=m, hbar=hbar)
print("x shape:", x_test.shape)
print("H shape:", H_test.shape)


x shape: (10,)
H shape: (8, 8)
